In [80]:
# Cargar librerías

import pandas as pd
import pywhatkit as kit
from datetime import datetime, timedelta
import time
import os
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import os
import locale

# Se establece la base de datos

In [54]:
# Se establece el idioma español

locale.setlocale(locale.LC_TIME, 'es_ES')

'es_ES'

In [55]:
# Cargar base de datos, agregar y modificar columnas

df = pd.read_csv("notificaciones.csv", encoding="latin1")
df['telefono'] = df['telefono'].apply(lambda x: f"+{x}")
df["titulo"] = df["victima_directa"].apply(lambda x: f"Notificación de Registro VD {x}")
df["fecha"] = pd.to_datetime(df["dia_notificacion"], format='%d de %B de %Y', errors='coerce', dayfirst=True)
df["fecha"] = df["fecha"].dt.strftime('%Y-%m-%d')
df["hora_notificacion"] = pd.to_datetime(df["hora_notificacion"], format="%H:%M:%S").dt.time
df["fecha_inicio"] = df.apply(lambda row: f"{row["fecha"]}T{row["hora_notificacion"]}", axis=1)
df["fecha_inicio"] = pd.to_datetime(df["fecha_inicio"])
df["fecha_fin"] = df["fecha_inicio"] + pd.Timedelta(minutes=59)

In [56]:
# Se vuelve a establecer el idioma default

locale.setlocale(locale.LC_ALL, '')

'C/UTF-8/C/C/C/C'

# Mandar whatsapps automáticos

In [3]:
# Definir función para envíar mensaje

def enviar_mensaje(numero, mensaje):
    try:
        kit.sendwhatmsg(numero, mensaje, tiempo_envio.hour, tiempo_envio.minute)  # Envía el mensaje en el tiempo programado
        print(f'Mensaje enviado a {numero}')
        time.sleep(10)  # Espera 2 segundos entre cada mensaje para evitar problemas con pywhatkit
    except Exception as e:
        print(f'Error al enviar mensaje a {numero}: {e}')


In [4]:
# Se define el tiempo de espera

now = datetime.now()
tiempo_envio = now + timedelta(seconds=60)
print("Esperando 25 segundos para cargar WhatsApp Web...")
time.sleep(30)

Esperando 25 segundos para cargar WhatsApp Web...


In [5]:
# Se itera sobre cada número para mandar el mensaje

for index, row in df.iterrows():
    mensaje = f"""
    Buenas tardes. Le escribe [Nombre Genérico], con cargo Enlace de Seguimiento a Fiscalías "A" en la CEAVI. (Esa información puede corroborarla en el sitio [URL Genérica])

    El motivo de este mensaje es para citarlos a una diligencia de NOTIFICACIÓN de su inscripción en el Registro de Víctimas de la Ciudad de México por los hechos victimizantes radicados en el expediente {row['numero_expediente']}, en relación con la víctima directa {row['victima_directa']}.

    Esta se llevaría a cabo el día {row['dia_notificacion']}, a las {row['hora_notificacion']} en las instalaciones de la CEAVI ubicadas en la [Dirección Genérica].

    Le informo que este número telefónico de contacto y demás datos personales fueron proporcionados en el respectivo Formato Único de Declaración mediante el cuál se realizó la solicitud de inscripción en el Registro.
    """
    
    try:
        enviar_mensaje(row['telefono'], mensaje)
        
        tiempo_envio += timedelta(seconds=60)
        
    except Exception as e:
        print(f'Error al enviar mensaje a {row["telefono"]}: {e}')

Error al enviar mensaje a +520000000000: Call Time must be Greater than Wait Time as WhatsApp Web takes some Time to Load!
In 50 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!
Mensaje enviado a +520000000000
In 34 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!
Mensaje enviado a +520000000000
In 34 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!
Mensaje enviado a +520000000000
In 35 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!
Mensaje enviado a +520000000000
In 35 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!
Mensaje enviado a +520000000000
In 34 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!
Mensaje enviado a +520000000000
In 35 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!
Mensaje enviado a +520000000000
In 34 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!
Mensa

# Agendar citas automáticas en Google Calendar

In [91]:
# Se accede a la API

SCOPES = ['https://www.googleapis.com/auth/calendar']
INVITADOS = ["...@gmail.com", "...@gmail.com"]

In [92]:
# Se crea la función para obtener el servicio

def obtener_servicio():
    creds = None
    if os.path.exists('Llaves/oauth.json'):
        flow = InstalledAppFlow.from_client_secrets_file('Llaves/oauth.json', SCOPES)
        creds = flow.run_local_server(port=0)
        with open('Llaves/llave.json', 'w') as token:
            token.write(creds.to_json())
    else:
        creds = Credentials.from_service_account_file('Llaves/llave.json', scopes=SCOPES)
    return build('calendar', 'v3', credentials=creds)

In [93]:
# Definir función para agendar el evento

def agendar_evento(servicio, evento):
    evento_creado = servicio.events().insert(calendarId='primary', body=evento).execute()
    print('Evento creado: %s' % (evento_creado.get('htmlLink')))

In [94]:
# Se accede al servicio de Google

servicio = obtener_servicio()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=936084092481-m92dmfpahg4igu51k3rc09n5ab7jo1rc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58695%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=NhhodjK2YiHKr2NefSFRFFkZGnQKH8&access_type=offline


In [95]:
# Se genera la función para agendar los eventos

for index, row in df.iterrows():
    evento = {
        'summary': row['titulo'],
        'location': 'México',
        'description': f'Notificación para {row["victima_directa"]}, expediente {row["numero_expediente"]}.',
        'start': {
            'dateTime': row['fecha_inicio'].isoformat(),
            'timeZone': 'America/Mexico_City',
        },
        'end': {
            'dateTime': row['fecha_fin'].isoformat(),
            'timeZone': 'America/Mexico_City',
        },
        'attendees': [{'email': email} for email in INVITADOS],
    }
    agendar_evento(servicio, evento)


Evento creado: https://www.google.com/calendar/event?eid=YXQ3cGNzaDllaGVtYm9ubWFwaXRoaDIwdjQgcmV2aWNlYXZpQGNkbXguZ29iLm14
Evento creado: https://www.google.com/calendar/event?eid=aWYxYnZzazJmODJlOWVlcTN1NHI1MWlrODggcmV2aWNlYXZpQGNkbXguZ29iLm14
Evento creado: https://www.google.com/calendar/event?eid=YWE4MWRnNzJjbW5uYWhma25rcWdhYWk2cWMgcmV2aWNlYXZpQGNkbXguZ29iLm14
Evento creado: https://www.google.com/calendar/event?eid=NTYybmFzbTg5aW1mOWFyNHFoNnRpYWJzNWcgcmV2aWNlYXZpQGNkbXguZ29iLm14
Evento creado: https://www.google.com/calendar/event?eid=Zzc4YXA4ZHBkOTN1cmpyazFhaGEzazR0ajggcmV2aWNlYXZpQGNkbXguZ29iLm14
Evento creado: https://www.google.com/calendar/event?eid=MTcwczg5MXV2cHBqNjd1aGkxZWZldDF1bG8gcmV2aWNlYXZpQGNkbXguZ29iLm14
Evento creado: https://www.google.com/calendar/event?eid=aHJiazBmbzc3Zm9vMGpsOWNhNjlzaGpqb28gcmV2aWNlYXZpQGNkbXguZ29iLm14
Evento creado: https://www.google.com/calendar/event?eid=NWNhbWJnanRjb2p2NjVta2Fiazg1cmJtamsgcmV2aWNlYXZpQGNkbXguZ29iLm14
Evento creado: https://w